<a href="https://colab.research.google.com/github/MartinAtoyan/MartinAtoyan-portfolio/blob/main/License_plate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# License Plate Detection using OpenCV and EasyOCR
This notebook detects license plates in images using OpenCV's Haar cascades and extracts text using EasyOCR.

In [ ]:
!pip install opencv-python imutils easyocr python-dotenv

In [ ]:
import os
import cv2
import numpy as np
import imutils
import easyocr
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get paths from .env
cascade_ref = os.environ.get("CASCADE")
images = os.environ.get("IMAGES")


In [ ]:
# Load Haar Cascade Classifier, cascade_ref is a pretrained Cascade model that detecting license plates.
cascade = cv2.CascadeClassifier(cascade_ref)

for image_dir in os.listdir(images):

    image_path = os.path.join(images, image_dir)
    # Read image
    image = cv2.imread(image_path)
    # Resize image
    image = imutils.resize(image, width=500)
    # Convert image to grayscale for better detection
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect plates with cascade
    # scalefactor control how much image is resized, each step of process
    # minNeighbors its a minimum rectangles for the same image/object.
    # It help to cancel false detections.
    plates = cascade.detectMultiScale(
        gray_image, scaleFactor=1.1, minNeighbors=5)

    # This loop iterates over the list plates, which contains
    # the coordinates of the detected license plates.
    # x, y coordinates upper left point of plate
    # w, h - width and height of plate
    for x, y, w, h in plates:
        # Create rectangle out of detected plate
        # x+w, y+w buttom right point of plate
        # (0, 255, 0) - color of rectangle
        # 2 - tickness of rectangle
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # Crop out the license plate area from image
        gray_plate = gray_image[y:y + h, x:x + w]

        # This function improves the contrast of the characters on the plate
        # 255 - maximum values of brightness for white pixels
        # cv2.ADAPTIVE_THRESH_GAUSSIAN_C - threshold calculation method
        # cv2.THRESH_BINARY - threshold processing type, pixels upper the
        # treshold become white, the rest become black
        # 11 - size of block, area around each pixel for calculating local threshold
        # 2 – constant, reduces background brightness
        processed = cv2.adaptiveThreshold(
            gray_plate, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY, 11, 2)

        # This function make a blur in image for reducing noises
        # (5, 5) - size of bluring, parameter must be odd
        # 1 - standart tone of bluring
        processed = cv2.GaussianBlur(processed, (5, 5), 1)

        # This function increase image size by 2
        # None - isn't new specific resolution
        # fx=2 - OX * 2
        # fy=2 - OY * 2
        processed = cv2.resize(processed, None, fx=2, fy=2)

        # Read the plate with english symbols
        reader = easyocr.Reader(['en'])
        plate_str = reader.readtext(processed)

        if plate_str:
            print(f'Detected Plate: {plate_str[0][1]}')

        # Show results
        cv2.imshow('Detected Plate', processed)
        cv2.imshow('Original Image', image)
        cv2.waitKey(0)

cv2.destroyAllWindows()

Sources

https://www.tutorialspoint.com/how-to-detect-license-plates-using-opencv-python

https://github.com/Maleehak/Car-number-plate-recognition-using-OpenCV

https://medium.com/augmented-startups/automatic-number-plate-recognition-with-easy-ocr-and-open-cv-6d121384cdc7

https://www.youtube.com/watch?v=73REqZM1Fy0

In [ ]:
.env.example
CASCADE='app\haarcascade_russian_plate_number.xml'
IMAGES='app\images'